In [ ]:
from math import pi
import numpy as np
from pde import plot_magnitudes,PDE,movie_scalar,UnitGrid,plot_magnitudes, CartesianGrid, MemoryStorage, ScalarField, plot_kymograph, plot_kymographs, FieldCollection, PlotTracker, UnitGrid, PDEBase, ExplicitSolver, FileStorage,VectorField
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LogNorm
from pde.tools.numba import jit
import matplotlib.backends.backend_pdf
import numba as nb
import pickle

In [ ]:
class PDE_1D_odd(PDEBase):
    def __init__(self, alpha=1, N=100, L=20):
        self.alpha = alpha
        self.N = N

    def evolution_rate(self, state, t=0):
        u, v = state
        rhs = state.copy()
        rhs[0] = v
        rhs[1] = (u+1j*self.alpha*u + v + np.abs(u)**2*u).laplace(bc='natural')
        return rhs

    def get_initial_state(self, array_u, array_v):
        grid = CartesianGrid([[0, L]], [self.N], periodic=True)
        return FieldCollection([ScalarField(grid, array_u,dtype=complex),ScalarField(grid, array_v,dtype=complex)])

    def _make_pde_rhs_numba(self, state):
        """ numba-compiled implementation of the PDE """
        alpha = self.alpha
        laplace = state.grid.get_operator("laplace", bc = "natural")
        gradient = state.grid.get_operator("gradient", bc = "natural")
        @nb.jit
        def pde_rhs(state_data, t):
            u = state_data[0]
            v = state_data[1]
            rate = np.empty_like(state_data)
            rate[0] = v
            rate[1] = laplace(u+1j*alpha*u+v+np.abs(u)**2*u)
            return rate

        return pde_rhs

In [ ]:
def ini_theory(alpha,L,N,n):
    x = np.linspace(0,L,N)
    amp = np.sqrt((alpha*L/(2*n*np.pi))**2-1)
    #analytical ss solution
    u = amp*np.exp(1j*(-2*n*np.pi/L * x - np.pi/2))
    v = -1j*alpha*u
    #constrain initial energy and momentum to be zero:
    u = u - np.average(u)
    v = v - np.average(v)    
    return u,v

def ini_ran(alpha,L,amp):
    x = np.linspace(0,L,N)
    u =amp* (np.random.rand(N)-0.5 +np.random.rand(N)*1j-0.5j)
    v=amp* (np.random.rand(N)-0.5 +np.random.rand(N)*1j-0.5j)
    #constrain initial energy and momentum to be zero:
    u = u - np.average(u)
    v = v - np.average(v)    
    return u,v

def get_initial_fromdata(file,pert=False,filter=False,t=-1):
    'generates an I.C. from the last timestep of a dataset'
    with open(file, 'rb') as output:
        dic = pickle.load(output)
        field = dic['field']
        alpha = dic['alpha']
        L = dic['L']
        u,v = dic['field']
        
    def perturb(u,v):
        unoise =(np.random.rand(N) -0.5 ) + 1j* (np.random.rand(N)-0.5)
        vnoise =(np.random.rand(N) -0.5 ) + 1j* (np.random.rand(N)-0.5)
        u += 2*unoise
        v += 2*vnoise
        return u,v
    
    def amplify(u,v):
        u = u*1.5
        v = v*1.5
        return u,v
    if pert:
        u,v= perturb(u,v)
    if filter:
        u,v = amplify(u,v)
    return u[t,:],v[t,:]     

In [ ]:

def sim1Dodd(alpha,N,L,pt,dt,ic):
    u,v = ic
    fields = []
    storage = MemoryStorage()
    trackers = ['progress'    , 'consistency'  ,     storage.tracker(interval=pt) ]
    eq = PDE_1D_odd(alpha=alpha,N=N,L=L)
    state = eq.get_initial_state(u, v) #initial field for u,v
    sol = eq.solve(state, t_range=tf,tracker=trackers,dt=dt)
    field =[]
    for j,i in storage.items():
        field.append(np.array(i.data))

    phi,dphi = np.asarray(field)[:,0,:],np.asarray(field)[:,1,:] 
    return phi,dphi

In [ ]:
#parameters
N=200
tf =1
pt = 0.1
L=10
alpha = 10
dt=5e-6
savefolder = 'data/'
savefile = 'test'
def runsim(alpha):
    u,v= ini_theory(alpha,L,N,n=8)
    phi,dphi= sim1Dodd(alpha,N,L,pt,dt,[u,v])
    filename = savefolder+savefile
    with open(filename, 'wb') as output:
        print('saved as:   ', filename)
        dic = {'phi':phi,'dphi':dphi,'L':L,'alpha':alpha,'N':N,'tf':tf,'pt':pt}
        pickle.dump(dic,output,pickle.HIGHEST_PROTOCOL)   
runsim(alpha)